In [8]:
import numpy as np
from fastapi import FastAPI, Form
import pandas as pd
from starlette.responses import HTMLResponse 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import re
import uvicorn


app = FastAPI()

@app.get('/predict', response_class=HTMLResponse)
def take_inp():
    return '''
        <form method="post">
        <input maxlength="28" name="text" type="text" value="Text Emotion to be tested" />
        <input type="submit" />'''
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)

INFO:     Started server process [10852]
INFO:uvicorn.error:Started server process [10852]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): normal olarak her yuva adresi (iletişim kuralı/ağ adresi/bağlantı noktası) için yalnızca bir kullanıma izin veriliyor
ERROR:uvicorn.error:[Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): normal olarak her yuva adresi (iletişim kuralı/ağ adresi/bağlantı noktası) için yalnızca bir kullanıma izin veriliyor
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\halil\anaconda3\lib\site-packages\uvicorn\server.py", line 147, in startup
    backlog=config.backlog,
  File "C:\Users\halil\anaconda3\lib\asyncio\base_events.py", line 1385, in create_server
    % (sa, err.strerror.lower())) from None
OSError: [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): normal olarak her yuva adresi (iletişim kuralı/ağ adresi/bağlantı noktası) için yalnızca bir kullanıma izin veriliyor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\halil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-8f42d33ca8f7>", line 21, in <module>
    uvicorn.run(app, host='127.0.0.1', port=8000)
  File "C:\Users\halil\anaconda3\lib\site-packages\uvicorn\main.py", line 386, in run
    server.run()
  File "C:\Users\halil\a

SystemExit: 1

INFO:     127.0.0.1:58360 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:58360 - "POST /predict HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:58493 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:63175 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51982 - "GET /predict HTTP/1.1" 200 OK


In [1]:
!pip install python-multipart

  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31667 sha256=81c608e453aa20c930113dd7b3a31a821e47238d93312a9c5a08c7bd62acd930
  Stored in directory: c:\users\halil\appdata\local\pip\cache\wheels\2c\41\7c\bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart


http://127.0.0.1:8000/predict 

In [1]:
import numpy as np
from fastapi import FastAPI, Form
import pandas as pd
from starlette.responses import HTMLResponse 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import re

def preProcess_data(text): #cleaning the data
    
    text = text.lower()
    new_text = re.sub('[^a-zA-z0-9\s]','',text)
    new_text = re.sub('rt', '', new_text)
    return new_text

app = FastAPI()

data = pd.read_csv('hepsiburada.csv')
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(data['Review'].values)



def my_pipeline(text):#pipeline
  text_new = preProcess_data(text)
  X = tokenizer.texts_to_sequences(pd.Series(text_new).values)
  X = pad_sequences(X, maxlen=59)
  return X


@app.get('/') #basic get view
def basic_view():
    return {"WELCOME": "GO TO /docs route, or /post or send post request to /predict "}



@app.get('/predict', response_class=HTMLResponse) #data input by forms
def take_inp():
    return '''<form method="post"> 
    <input type="text" maxlength="59" name="text" value="Duygu Dedektörü"/>  
    <input type="submit"/> 
    </form>'''



@app.post('/predict') #prediction on data
def predict(text:str = Form(...)): #input is from forms
    clean_text = my_pipeline(text) #cleaning and preprocessing of the texts
    loaded_model = tf.keras.models.load_model('duygu.h5') #loading the saved model
    predictions = loaded_model.predict(clean_text) #making predictions
    sentiment = int(np.argmax(predictions)) #index of maximum prediction
    probability = max(predictions.tolist()[0]) #probability of maximum prediction
    if probability<0.5: #assigning appropriate name to prediction
        t_sentiment = 'Olumsuz'
    elif probability>0.5:
        t_sentiment = 'Olumlu'
    
    return { #returning a dictionary as endpoint
        "Girdiğiniz cümle": text,
        "Duygu Durumu": t_sentiment,
        "Duygu Yüzdesi": probability
    }

In [2]:
import uvicorn
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)

RuntimeError: This event loop is already running

INFO:     Started server process [12024]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54138 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:54146 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:54147 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:54151 - "POST /predict HTTP/1.1" 200 OK
